# Notebook 4: Projecting Injections

In this notebook we will explore how we can use GravyFlow GPU functions to project injections onto the GPU with physically in a physically realistic fashion. As ususal we wills start by performing the neccisary imports:

In [2]:
# Built-in imports
from typing import List
from pathlib import Path

# Dependency imports: 
import numpy as np
import tensorflow as tf
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# Import the GravyFlow module.
import gravyflow as gf

2025-02-11 08:12:26.228019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739283146.247665  611281 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739283146.253702  611281 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 08:12:26.274726: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In order to demonstrate projection onto a detector network, we will need example waveforms. We can use the RippleGenerator we learned about in notebook 3 to generate example waveforms:

In [3]:
# Create a waveform generator to generate example IMRPhenomD waveforms:
phenom_d_generator : gf.WaveformGenerator = gf.RippleGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=0.0
)
phenom_d_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(phenom_d_generator)

# Use the TensorFlow environment 'env' created earlier with gf.env()
with gf.env():

    # Generate one exammple waveform with the generator:
    phenom_d_injection, _, _ = next(phenom_d_injection_generator(num_examples_per_batch=1))

2025-02-11 08:12:36,058 - INFO - Available GPUs: ['5']
2025-02-11 08:12:36.318983: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1739283156.319442  611281 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3000 MB memory:  -> device: 5, name: Tesla V100-SXM2-16GB, pci bus id: 0000:86:00.0, compute capability: 7.0
2025-02-11 08:12:36,376 - INFO - Visible GPUs after restriction: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_devi

5


I0000 00:00:1739283156.782946  611281 service.cc:148] XLA service 0xb57e410 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739283156.782997  611281 service.cc:156]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
I0000 00:00:1739283156.796353  611281 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1739283156.855957  611281 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


## Projecting Injections onto Multiple Detectors

GravyFlow uses the `gf.Network` class to handle networks of virtual gravitational wave detectors.

`gf.Network` has the following initilisation parameters:

- `parameters` : `Union[List[IFO], Dict]`, Required
  > There are two ways to define the detector network when intilising a `gf.Network` object. A list of interferometers already hard coded in to GravyFlow, or a dictionary containing custom interferometers.

- `seed` : `int` = `None`
  > The seed for the random number generators used when projecting injections. If set to None, the seed from `gf.Defaults` will be used.

First, we will initlise a detector object with a list of detectors already hard coded into GravyFlow:

In [4]:
# Generate a gf.Network object that constists of the Livinston (L1), Hanford (H1), and Virgo (V1) detectors.
network : gf.Network = gf.Network(
    parameters=[gf.IFO.L1, gf.IFO.H1, gf.IFO.V1]
)

2025-02-11 08:12:38.575701: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


With this `gf.Network` object, we can project the injections that we previously generated, using the `project_wave` class function, which has the following arguments:

- `strain` : `tf.Tensor`, Required
  > A `tf.Tensor` containing batches of both polarizations of raw strain of the injection that you wish to project.

- `sample_rate_hertz` : `Optional[float]` = `None`, Optional
  > The sample rate in Hertz of the input strain. Defaults to the value sample rate set in `gf.Defaults`.

- `right_ascension` : `Optional[Union[tf.Tensor, List[float], float]]` = `None`, Optional
  > The right ascesnion of the simulated source of the gravitational wave, which will be used for the projection of the injection. By default, the sources will be randomly distributed across the sky.

- `declination` : `Optional[Union[tf.Tensor, List[float], float]]` = `None`, Optional
  > The declination of the simualted source of the gravitational wave, which will be used for the projection of the injection. By default, the sources will be randomly distributed across the sky.

-  `polarization` : `Optional[Union[tf.Tensor, List[float], float]]` = `None`, Optional
  > The polarizaton of the simulated source of the gravitational wave, which will be used for the projection of the injection. By default, the sources will be randomly distributed across the sky.

Let's project the world using a toy value of 0.0 for all of the three sky location and polarization parameters: `right_ascension`, `declination`, `polarization`.

In [5]:
with gf.env():
    # Project the waveform contained in phenom_d_injection onto the detectors 
    # defined in network.projection_wave:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0],
        right_ascension=0.0,
        declination=0.0,
        polarization=0.0
    )

2025-02-11 08:12:39,221 - INFO - Available GPUs: ['5']
2025-02-11 08:12:39,223 - INFO - Visible GPUs after restriction: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]


5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: name node, input, outside of any statement?
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


We can then plot the output of the waveform projection, the projection will create one channel for each detector, and will no longer contain both polarizations as they have been reduced in the projection.

In [6]:
# Since we are only projecting one injection, extract the first injection:
projected_injection : tf.Tensor = projected_injections[0]

# Plot the three channel output of the projection.
projection_layout : List = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Projecting from a random Sky Location and Polarization:

Rather than inputing a specific sky localisation and polarization and direction. The default behaviour of `project_wave` is to project the waveform from a random sky direction and polarization.

We can see that below:

In [7]:
# Random direction and polarisation
with gf.env():
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]
projection_layout : List = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2025-02-11 08:12:40,563 - INFO - Available GPUs: ['5']
2025-02-11 08:12:40,565 - INFO - Visible GPUs after restriction: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]


5


Loading BokehJS ...

## Adding custom detectors

We can also add custom detectors using a dictionary, which we can see in the example below:

In [8]:
# Adding Custom Detectors:

# Create from dictionary:
network = gf.Network({
    "longitude_radians" : [np.pi/2, np.pi/4], 
    "latitude_radians" : [-np.pi/4, np.pi/6],
    "y_angle_radians" : [(2*np.pi/3), np.pi], 
    "x_angle_radians" : None, 
    "height_meters" : [0.0, 0.0],
    "x_length_meters" : [4000.0, 10000.0],
    "y_length_meters" : [4000.0, 10000.0]
})

# Random direction and polarisation
with gf.env():
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2025-02-11 08:12:41,410 - INFO - Available GPUs: ['5']
2025-02-11 08:12:41,412 - INFO - Visible GPUs after restriction: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]


5


Loading BokehJS ...

## Load Detector From Config

We can also load a custom network from a `.json` file, as in the example below:

In [9]:
# Define injection directory path:
example_network_directory : Path =  Path("./example_configs/example_network.json")

network = gf.Network.load(example_network_directory)

# Random direction and polarisation
with gf.env():
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2025-02-11 08:12:42,052 - INFO - Available GPUs: ['5']
2025-02-11 08:12:42,053 - INFO - Visible GPUs after restriction: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]


5


Loading BokehJS ...

## Incoherent Injections

We can generate incoherent injections by using a `gf.IncoherentGenerator` composed of other waveform generators, which allows us to use a different waveform in each detector. Note that if we do this we must be carefull to use the same number of component waveforms as there are detectors in the network we use to project with.

In [10]:
wnb_generator_a : gf.WaveformGenerator = gf.WNBGenerator(
    duration_seconds=0.7,
    min_frequency_hertz=50.0,
    max_frequency_hertz=100.0
)
phenom_d_generator_a : gf.WaveformGenerator = gf.RippleGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=10.0
)
phenom_d_generator_b : gf.WaveformGenerator = gf.RippleGenerator(
    mass_1_msun=10.0,
    mass_2_msun=10.0,
    inclination_radians=20.0
)

incoherent_generator : gf.InjectionGenerator  = gf.IncoherentGenerator(
    [wnb_generator_a, phenom_d_generator_a, phenom_d_generator_b]
)

incoherent_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(incoherent_generator)    

Finally, we can plot these incoherent injections:

In [11]:
network = gf.Network([gf.IFO.L1, gf.IFO.H1, gf.IFO.H1])

with gf.env():
    incoherent_injections, _, _ = next(
        incoherent_injection_generator(num_examples_per_batch=1)
    )        

    projected_injections : tf.Tensor = network.project_wave(
        incoherent_injections
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2025-02-11 08:12:42,543 - INFO - Available GPUs: ['5']
2025-02-11 08:12:42,545 - INFO - Visible GPUs after restriction: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]


5


Loading BokehJS ...